In [ ]:
##-----degraded-----##
# !pip install -Iv gym==0.17.1
# !apt-get install python-opengl -y
# !apt install xvfb -y
# !pip install gym[atari]
# !pip install pyvirtualdisplay
# !conda install piglet
# !pip install pystan
# !conda install swig
# #!pip install box2d-py
# !pip install box2d-py
# !pip install gym[box2d]

In [ ]:
# Upload 'HC ROMS.zip' and 'ROMS.zip' online.
!python -m atari_py.import_roms .
import gym
env = gym.make('PongNoFrameskip-v4')
print(env.observation_space.shape)

copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ROMS/Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/assault.bin
copying asterix.bin from ROMS/Asterix (AKA Taz) (07-27-1983) (Atari, Jerome Domurat, Steve Woita) (CX2696) (Prototype).bin to /usr/local/lib/python3.7/dist-packages/a

In [ ]:
#(i) Create a folder called 'common'
# Then uploaad the .py file 'old_atari_wrappers.py' to 'common'
# (ii) Copy the main code from .py file to this block and finish the code.
# NOTE: COLAB can maximlly connect with 12 hrs, and you will need to check this tab from time to time to see if it is still connected or let you check whether you are ROBOT.

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed May 12 22:46:13 2021
This task requires some efforts and is evlauted as the highest difficulty-level so far. Not really difficult, but a lot of details.
You can take the code from Bonus task 1 to use here.
@author: hongh
"""
import math, random
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
import itertools
from common.old_atari_wrappers import wrap_pytorch, make_atari, wrap_deepmind
import matplotlib.pyplot as plt
            

def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        # To do: using torch.nn.init.kaiming_normal_() to initialize m.weight , choose the proper nonlinearity 
        # m.bias can be intialized to be zero, using torch.nn.init.zeros_()
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_in',nonlinearity='relu')
        torch.nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        # To do: Since we use the leakyReLU with slope of 0.2, you need to choose appropriate nonlinearity slope.
        # bias and weights need to be intialized respectively.
        torch.nn.init.kaiming_normal_(m.weight, mode='fan_in', a=0.2, nonlinearity='leaky_relu')
        torch.nn.init.zeros_(m.bias)


class QNetwork(nn.Module):    
    def __init__(self, input_shape, num_actions):
        super(QNetwork, self).__init__()
        self.input_shape = input_shape
        self.num_actions = num_actions

        # build your network according to the paper "Playing Atari with Deep Reinforcement Learning"
        # To prevent "dead neurons" of ReLU activation, you could use instead Leakyrelu with the negative slope of 0.2 after the fully-connected layer with 256 neurons.       
        # Hint: nn.sequential() , nn.Conv2d(), nn.Linear()....
        
        # To do: self.features only contains the 3 convolutional blocks and their nonliner activation functions
        self.features = nn.Sequential(
            nn.Conv2d(4, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.ReLU()
        )
        # To do: apply the weight initialization scheme on self.features using the defined function init_weights.
        self.features.apply(init_weights)

        
        
        # To do: Initialize fully connected layer () , then followed by leaklyReLU with slope 0.2, (We don't use ReLU here), and last linear layer
        self.fc1 = nn.Linear(64*7*7, 256)
        self.fc1.apply(init_weights)
        self.leaky_relu = nn.LeakyReLU(0.2)
        self.fc2 = nn.Linear(256, num_actions)
        
        
        
    def forward(self, state):
        # To do: before forwarding the state, the state should be normalized between [0,1], the original pixel range is [0,255]
        state = torch.div(state, 255)
        state = state.cuda()
        
        feats = self.features(state)
        fc1 = self.fc1(feats.view(feats.size(0),-1))
        leaky_relu = self.leaky_relu(fc1)

        return self.fc2(leaky_relu).cuda()

    def feature_size(self):
        with torch.no_grad():
            return self.features(torch.zeros(1, *self.input_shape)).view(1, -1).size(1)
        


In [ ]:


class LearningAgent: 
    def __init__(self, **kwargs):      
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using {} for training the algorithm.".format(self.device))        
        self.env = kwargs.get('training_env',None)
        self.gamma = kwargs.get('gamma', 0.99)
        self.hard_update_interval = kwargs.get('target network update interval', 10000)
        
        self.update_step = 0
        
        # ---- initialize networks ----
        self.q_net1 = QNetwork(env.observation_space.shape, env.action_space.n).to(self.device)
        self.target_q_net1 = QNetwork(env.observation_space.shape, env.action_space.n).to(self.device)


        # To do: initialize optimizers using Adam optimizer, learning rate is in kwargs.
        self.q1_optimizer = optim.Adam(self.q_net1.parameters(), kwargs.get('learning rate', 1e-4))
        # To do: initialize the loss function (Huberloss) , keywords:  torch.nn.SmoothL1Loss
        # Note : here the reduction scheme should be set to be 'none', different from previous one. It is needed for incorporating IS-weights.
        self.criterion = nn.SmoothL1Loss(reduction='none')
        
        # IMPORTANT: In our implementation, we use sum tree, whose root node number must be 2^n. 
        self.replay_buffer = PER(kwargs.get('memory size',1048576), self.env.observation_space.shape)
        


    def act(self, state):
        # To do: implement epsilon-greedy, where action is the index (just a number) , not a array/list/tensor.
        # Hint: to speed up, use with torch.no_grad():
        state = torch.tensor(state)
        if state.size(0) == 4:
          state = state.unsqueeze(0).to(self.device)

        with torch.no_grad():
          # Epsilon-Greedy action selection
          if np.random.random() > self.epsilon: # Exploit
            action = self.q_net1.forward(state)
            act = torch.argmax(action).item()
          else:  # Explore
            act = env.action_space.sample()
        return act


    def test_act(self, state):
        # To do: implement greedy policy, where action is the index (just a number) , not a array/list/tensor.
        # Hint: to speed up, use with torch.no_grad():
        state = torch.tensor(state)
        if state.size(0) == 4:
          state = state.unsqueeze(0).to(self.device)

        with torch.no_grad():
          action = self.q_net1.forward(state)
          act = torch.argmax(action).item()
        return act
    
    
   
    def update(self, batch_size):       
        # To do: implement DDQN with prioritized experience replay, now the loss is different due to IS weights.
        self.q_net1.train()
        
        transitions, idxs, weights = self.replay_buffer.sample(batch_size)        
        states, actions, rewards, next_states, dones = transitions
        weights = torch.FloatTensor(weights).to(self.device)
        states = torch.FloatTensor(states).to(self.device)
        actions = torch.LongTensor(actions).to(self.device)
        rewards = torch.FloatTensor(rewards).to(self.device)
        next_states = torch.FloatTensor(next_states).to(self.device)
        dones = torch.FloatTensor(dones).to(self.device)
        
        #DDQN.....
        with torch.no_grad():
          #DDQN
          act_idx = self.q_net1(next_states).argmax(1).unsqueeze(1)
          next_state_act_vals = self.target_q_net1(next_states).gather(1, act_idx)
          y_i = (1 - dones)*(next_state_act_vals.squeeze(1) * self.gamma) + rewards#DDQN

        state_action_values = self.q_net1(states).gather(1, actions.unsqueeze(1)) #self.q_net1(states).gather(1, act_idx)


        # To do:  q loss using self.criterion
        q1_loss = self.criterion(state_action_values, y_i.detach().unsqueeze(1)).squeeze()     
          
        # For PER algorithm code line 12,  To do: convert q1_loss from (Cuda) tensor to numpy, 
        td_error_abs = np.abs(q1_loss.detach().cpu().numpy()) + 1e-7#rewards.detach().cpu().numpy() + self.gamma*q1_loss.detach().cpu().numpy() + 1e-7 # adding 1e-7 to avoid singulrities.
        # To do: incorporate IS_weight for each sample and then taking the mean.
        #q1_loss = np.mean((state_action_values, y_i.detach().unsqueeze(1)) * weights)#np.mean(np.sqrt((state_action_values.detach().cpu().numpy() - y_i.unsqueeze(1).detach().cpu().numpy()) ** 2) * weights.detach().cpu().numpy())#...
        
        
        # update priority for trained samples
        for idx, td_error in zip(idxs, td_error_abs):
            self.replay_buffer.update_priority(idx, td_error)

        q1_loss = (q1_loss*weights).mean()
        # update q networks        
        self.q1_optimizer.zero_grad()
        q1_loss.backward()
        self.q1_optimizer.step()

        # --------To do: Perform Hard update of target networks every self.hard_update_interval-----------     
        if self.update_step % self.hard_update_interval == 0:    
          for target_param, param in zip(self.target_q_net1.parameters(), self.q_net1.parameters()):
            target_param.data.copy_(param.data)
        
        self.update_step += 1
        
        # just for monitoring Q
        if self.update_step % 1000 == 999:
            print("Q loss: {}".format(q1_loss))



In [ ]:


class SumTree:
    
    write = 0

    def __init__(self, capacity):
        self.capacity = capacity
        self.data = np.zeros(capacity, dtype=object) # storing experience
        self.tree = np.zeros(2 * capacity - 1) # storing pripority + parental nodes. If you have n elements in the bottom, then you need n + (n-1) nodes to construct a tree.        
        self.n_entries = 0
        self.overwrite_start_flag = False # record whether N_entry > capacity
        
    # update to the root node
    def _propagate(self, idx, change):
        parent = (idx - 1) // 2
        self.tree[parent] += change
        if parent != 0:
            self._propagate(parent, change)

    # find sample on leaf node
    def _retrieve(self, idx, s):
        left = 2 * idx + 1
        right = left + 1
        if left >= len(self.tree):
            return idx
        if s <= self.tree[left]:
            return self._retrieve(left, s)
        else:
            return self._retrieve(right, s - self.tree[left])


    def total(self):
        return self.tree[0]

    # store priority and sample
    def add(self, p, data):
        idx = self.write + self.capacity - 1 # starting write from the first element of the bottom layer,
        self.data[self.write] = data
        self.update(idx, p)
        self.write += 1
        if self.write >= self.capacity:
            self.write = 0
            self.overwrite_start_flag = True
        if self.n_entries < self.capacity:
            self.n_entries += 1


    # update priority
    def update(self, idx, p):
        change = p - self.tree[idx]
        self.tree[idx] = p
        self._propagate(idx, change)

    # get priority and sample
    def get(self, s):
        idx = self._retrieve(0, s)
        dataIdx = idx - self.capacity + 1
        return (idx, self.tree[idx], self.data[dataIdx])   
    
    def max_priority(self):
        return np.max(self.tree[self.capacity-1:])

    # ---- modified : to avoid min_prob being overwritten, causing a potential change in max_wi --> Q-loss suddenly changes/rescales --> unstable training  ----
    def min_prob(self):
        ''' To do: 
            return: p_min, the minimal of P(j) in the replay buffer, see line 9 in algorithm in the original paper https://arxiv.org/pdf/1511.05952.pdf.
            p_min is used to determine max w_{i}
        '''
        #print(self.tree)
        
        #nonzero_idxs = np.where(self.tree != 0.0)
        #values = self.tree[nonzero_idxs]
        p = self.tree[self.capacity-1:]

        if self.overwrite_start_flag == False:        
            p_min = np.min(p[p > 0])#np.min(self.tree[self.capacity-1:])
            self.p_min_history = p_min
        elif self.overwrite_start_flag == True: 
            p_min = min(np.min(p[p > 0]),self.p_min_history)
            self.p_min_history = min(p_min, self.p_min_history)
        return p_min



In [ ]:

class PER(object):
    '''Here we only use  proportional prioritization , not rank-based prioritiation, which is reported to be of better performance.
    '''
    def __init__(self, max_size, shape, alpha=0.6, beta=0.4):
        self.sum_tree = SumTree(max_size)
        self.alpha = alpha
        self.beta = beta
        self.beta_increment = (1.0 - beta)/100000.
        self.current_length = 0
        self.frame_stack = shape[0]
        self.state_dim = shape
        
        
    def push(self, state, action, reward, next_state, done):
        # To do: store the priority
        if self.current_length == 0: # The first sample with priority = 1
            priority = 1.0  
        else: # maximal priority
            priority = self.sum_tree.max_priority()
        self.current_length = self.current_length + 1
        # To Note: we directly priority = td_error ^ self.alpha into the node, which is befneficial for sampling later.
       

        experience = (state, np.array([action]), np.array([reward]), next_state, done)
        self.sum_tree.add(priority, experience)


    def sample(self, batch_size):
        # Perform a Stochastic universal sampling by roulette wheel, which reduces the variance.
        batch_idx, batch, priorities = [], [], []
        segment = self.sum_tree.total() / batch_size
        p_sum = self.sum_tree.tree[0]
    
        for i in range(batch_size): 
            a = segment * i
            b = segment * (i + 1)
            s = random.uniform(a, b)
            idx, p, data = self.sum_tree.get(s)
            batch_idx.append(idx)
            batch.append(data)
            priorities.append(p)
            
        # To do: Compute the IS_weights for each sampled experience.           
        prob = priorities / p_sum # a 1-D array with the length of batch size
        IS_weights = np.power((self.current_length * prob), -self.beta) # a function of prob
        max_weight = np.power((1/self.sum_tree.n_entries) * (1/(self.sum_tree.min_prob()/p_sum)), self.beta) # Hint : you will need to call self.sum_tree.min_prob()
        IS_weights /= max_weight # should be of the same shape as 'prob'
               
        state_batch = np.empty((batch_size,*self.state_dim))
        action_batch = np.empty((batch_size))
        reward_batch = np.empty((batch_size))
        next_state_batch = np.empty((batch_size,*self.state_dim))
        done_batch = np.empty((batch_size))   

        for i,transition in enumerate(batch):
            state, action, reward, next_state, done = transition
            state_batch[i] = state
            action_batch[i] = action
            reward_batch[i] = reward
            next_state_batch[i] = next_state
            done_batch[i] = done

        # To do: update beta.
        self.beta = min(1, self.beta + self.beta_increment)
        return (state_batch, action_batch, reward_batch, next_state_batch, done_batch), batch_idx, IS_weights

    def update_priority(self, idx, td_error):
        # Slightly different from original code, we save the exponeniated priority as follows to the sum tree, which is easier to take samples according to the probability later.
        # To do: priority = td_error ^ self.alpha
        priority = np.power(td_error, self.alpha)
        self.sum_tree.update(idx, priority)

    def __len__(self):
        return self.current_length


In [ ]:





def learn(agent, **kwargs):   
    update_step = 0
    train_start = kwargs.get('number of pre-interactions to start training', 50000)
    batch_size = kwargs.get('batch size', 32)
    try:
        max_steps = env._max_episode_steps# 
    except:
        max_steps = kwargs.get('max steps per episode', 27500)   
    max_episodes = kwargs.get('max episodes', 1000000)   
    update_every_n_step = kwargs.get('update every n steps', 4)   
    n_frame = 0
    running_100_mean = np.zeros(100)    
    epsilon_start = 1
    epsilon_final = 0.1
#    epsilon_decay = 10000
    
    epi_returns = [] # Important: you will save this statistics for plotting
    test_phase_avg_returns = []

    for episode in range(max_episodes):    
                
        state = env.reset()
        epi_return = 0
        for step in itertools.count():
            # Implement linearly-decaying epsilon
            agent.epsilon = max(epsilon_final, epsilon_start - (epsilon_start - epsilon_final)* n_frame/500000. )
            action= agent.act(state)                            
            next_state, reward, done, _ = env.step(action)            
            n_frame+=1        
            # Push the experience into replay buffer                                                              
            agent.replay_buffer.push(state, action, reward, next_state, done)
            epi_return += reward

            # To do: write an if condition that the training starts after the agent collects enough experience () and performs an update every 4 interactions.
            # Useful variable :  train_start , update_every_n_step , n_frame
            if (len(agent.replay_buffer) > train_start) and ((n_frame % update_every_n_step) == 0): 
                agent.update(batch_size)
                update_step += 1
            state = next_state
            
            if done == True or step >= max_steps:
                running_100_mean[episode%100] = epi_return
                epi_returns.append(epi_return)
                print('Episode : {} , episodic length : {}, return: {}, exp_count :{}, averged returnover past 100 episodes : {}'.format(episode , step, epi_return, n_frame, np.mean(running_100_mean[:min(100,episode+1)]))) 
                break

        #---------------To Do: Save the satistics of ----------------
        if episode%50 == 49: # you can change here
            epi_returns_np = np.array(epi_returns)
            #To do: save the statistics here for plotting 
            np.save("epi_returns_PER.npy", epi_returns_np)

        #----------------------------Testing-----------------------
        if episode%100 == 99:
            test_returns = np.zeros(15)
            print ('-------start Testing--------')
            epi_return = 0       
            state = test_env.reset()  
            for i in range(15): # test for 15 episodes
                state = test_env.reset() 
                for t in range(max_steps):
                    # env.render()
                    action = agent.test_act(state)
                    next_state, reward, done, _ = test_env.step(action)
                    state = next_state
                    epi_return += reward                        
                    if done or (t == max_steps - 1):
                        test_returns[i] = epi_return
                        epi_return = 0
                        break         
            test_phase_avg_returns.append(np.mean(test_returns))           
            print('Testing result, Returns: {}'.format(test_returns))
            test_phase_avg_returns_np = np.array(test_phase_avg_returns)
            # ---IMPORTNT: To do: save the statistics here for plotting ---
            np.save("test_phase_avg_returns.npy", test_phase_avg_returns_np)


if __name__ == '__main__':  
    
    env_id = "PongNoFrameskip-v4" # you could change to other games, but only use 'NoFrameskip-v4' version. Some games e.g., spaceinvaders require changing the code in wrapper, i.e., frame_skip =3
    env = make_atari(env_id)
    env = wrap_deepmind(env)
    env = wrap_pytorch(env) # Note the env returns the unnormalized pixel [0,255].
    test_env = make_atari(env_id)
    test_env = wrap_deepmind(test_env, episode_life=False, clip_rewards=False)
    test_env = wrap_pytorch(test_env)#wrap_pytorch_with_normalization(env)
    
    # Hint: you could check the shape and content of the state, it is not yet normlized between [0,1]
    obs = env.reset()
    print(obs.shape)
    for i in range(obs.shape[0]):
        plt.imshow(obs[i])
    
    random_seed = None
    if random_seed:
        print("Random Seed: {}".format(random_seed))
        torch.manual_seed(random_seed)
        env.seed(random_seed)
        np.random.seed(random_seed)  
        random.seed(random_seed)
        
    # DQN params
    config = {}
    config.update({'gamma': 0.99})
    config.update({'learning rate': 1e-4})
    config.update({'memory size': 262144}) # If you use Prioritized EXP, size must be 2^n. 1M frame memory buffer takes around 8GB in RAM. For local computer, you could use higher memory size.
    config.update({'target network update interval': 3000}) # every 3000 training steps (not interaction steps), update the target model.
    config.update({'max episodes': 1000000}) # you don't need to run that long, but just keep running.
    config.update({'max steps per episode': 27500}) 
    config.update({'batch size': 32}) 
    config.update({'update every n steps': 4})
    config.update({'number of pre-interactions to start training': 50000})
    config.update({'training_env': env})
    config.update({'testing_env': test_env})
    #  agent
    agent = LearningAgent(**config)
    
    # train
    learn(agent, **config)


(4, 84, 84)
Using cuda for training the algorithm.
Episode : 0 , episodic length : 848, return: -20.0, exp_count :849, averged returnover past 100 episodes : -20.0
Episode : 1 , episodic length : 1137, return: -18.0, exp_count :1987, averged returnover past 100 episodes : -19.0
Episode : 2 , episodic length : 1015, return: -19.0, exp_count :3003, averged returnover past 100 episodes : -19.0
Episode : 3 , episodic length : 936, return: -21.0, exp_count :3940, averged returnover past 100 episodes : -19.5
Episode : 4 , episodic length : 880, return: -20.0, exp_count :4821, averged returnover past 100 episodes : -19.6
Episode : 5 , episodic length : 900, return: -21.0, exp_count :5722, averged returnover past 100 episodes : -19.833333333333332
Episode : 6 , episodic length : 894, return: -21.0, exp_count :6617, averged returnover past 100 episodes : -20.0
Episode : 7 , episodic length : 1062, return: -20.0, exp_count :7680, averged returnover past 100 episodes : -20.0
Episode : 8 , episodi